In [16]:
!pip install -q ultralytics timm

# Download files

In [17]:
import gdown

#download model
gdown.download_folder(id='1QVWjX8Aa2VvIbXjDY4DyqcV_Uqw6myX7', output="model", quiet=False, use_cookies=False)

#download test ds
gdown.download_folder(id='1LZIhiV9l82W4fNpfaBHyGvh8XBX6SoU2', output="test_folder", quiet=False, use_cookies=False)

Retrieving folder contents


Processing file 1vA70lsUOr-TYG7337eQ7Z-X4J7WbLvVD [TMP] cnn kuih-scratch.pt
Processing file 1_10GTTSqYCDbFgEdzRyU3YUXgaGc8aqN [TMP] labels.txt
Processing file 11sJJF6_W7UbGbskoFSBZ0dMTzkHNCLBe [TMP] vit_kuih_balanced_epoch42_loss0.001_0.9114acc.pth


Retrieving folder contents completed
Building directory structure
Building directory structure completed
Downloading...
From (original): https://drive.google.com/uc?id=1vA70lsUOr-TYG7337eQ7Z-X4J7WbLvVD
From (redirected): https://drive.google.com/uc?id=1vA70lsUOr-TYG7337eQ7Z-X4J7WbLvVD&confirm=t&uuid=041217d8-4811-4b6d-af08-4927c2546b56
To: c:\Users\ochon\OneDrive - student.newinti.edu.my\NAIC_submission\naic\model\[TMP] cnn kuih-scratch.pt
100%|██████████| 125M/125M [00:04<00:00, 25.1MB/s] 
Downloading...
From: https://drive.google.com/uc?id=1_10GTTSqYCDbFgEdzRyU3YUXgaGc8aqN
To: c:\Users\ochon\OneDrive - student.newinti.edu.my\NAIC_submission\naic\model\[TMP] labels.txt
100%|██████████| 124/124 [00:00<?, ?B/s] 
Downloading...
From (original): https://drive.google.com/uc?id=11sJJF6_W7UbGbskoFSBZ0dMTzkHNCLBe
From (redirected): https://drive.google.com/uc?id=11sJJF6_W7UbGbskoFSBZ0dMTzkHNCLBe&confirm=t&uuid=0f361c9f-c32a-47d3-a8be-ac879e735507
To: c:\Users\ochon\OneDrive - student.newinti.

Retrieving folder 1qO-cQtRdp-hLu6LMQ68OEv0zA791puJO test_images
Processing file 1cxftTp4A1_13gWe4QeAlhK9AcGzIDBEH Kek Lapis.jpg
Processing file 1aqVkDJiANm0MqLeTJALC1RuKZgt0IlI4 Kuih Kaswi Pandan.jpg
Processing file 11K7Hc9nt3Qllw4lG4ajxineVHvXKmuY6 Kuih Ketayap.jpg
Processing file 1qTcPAX5eSPLkAkhlyGfgooDXHZPFoJOM Kuih Lapis.jpg
Processing file 17CPxIkfU_tecxiF1-G4nXTD0Vv_1biB- Kuih Seri Muka.png
Processing file 1MEhhjAoBK66wtj6Xs1orsrA2vR9qLjhR Kuih Talam.jpg
Processing file 1KmI2JRpz4DnCAmPbCWoK31BJBH4ax8i7 Kuih Ubi Kayu.jpg
Processing file 1QFZdtM4ok-vH0kPgA7gdZQzT1aSppWcF Onde-Onde.jpg
Processing file 12ufiwhl6whVp33F05RLiDRieRSN13XlP test_labels.txt


Retrieving folder contents completed
Building directory structure
Building directory structure completed
Downloading...
From: https://drive.google.com/uc?id=1cxftTp4A1_13gWe4QeAlhK9AcGzIDBEH
To: c:\Users\ochon\OneDrive - student.newinti.edu.my\NAIC_submission\naic\test_folder\test_images\Kek Lapis.jpg
100%|██████████| 1.27M/1.27M [00:00<00:00, 1.57MB/s]
Downloading...
From: https://drive.google.com/uc?id=1aqVkDJiANm0MqLeTJALC1RuKZgt0IlI4
To: c:\Users\ochon\OneDrive - student.newinti.edu.my\NAIC_submission\naic\test_folder\test_images\Kuih Kaswi Pandan.jpg
100%|██████████| 418k/418k [00:00<00:00, 9.18MB/s]
Downloading...
From: https://drive.google.com/uc?id=11K7Hc9nt3Qllw4lG4ajxineVHvXKmuY6
To: c:\Users\ochon\OneDrive - student.newinti.edu.my\NAIC_submission\naic\test_folder\test_images\Kuih Ketayap.jpg
100%|██████████| 373k/373k [00:00<00:00, 6.78MB/s]
Downloading...
From: https://drive.google.com/uc?id=1qTcPAX5eSPLkAkhlyGfgooDXHZPFoJOM
To: c:\Users\ochon\OneDrive - student.newinti.edu

['test_folder\\test_images\\Kek Lapis.jpg',
 'test_folder\\test_images\\Kuih Kaswi Pandan.jpg',
 'test_folder\\test_images\\Kuih Ketayap.jpg',
 'test_folder\\test_images\\Kuih Lapis.jpg',
 'test_folder\\test_images\\Kuih Seri Muka.png',
 'test_folder\\test_images\\Kuih Talam.jpg',
 'test_folder\\test_images\\Kuih Ubi Kayu.jpg',
 'test_folder\\test_images\\Onde-Onde.jpg',
 'test_folder\\test_labels.txt']

# Load CNN model

In [18]:
from ultralytics import YOLO

cnn = YOLO("model/[TMP] cnn kuih-scratch.pt")

# Load ViT

In [19]:
import timm
import torch

vit = timm.create_model(
    'eva02_base_patch14_224.mim_in22k', pretrained=False, num_classes=8)

vit.load_state_dict(torch.load("model/[TMP] vit_kuih_balanced_epoch42_loss0.001_0.9114acc.pth"))

vit.eval().cuda()


Eva(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(3, 768, kernel_size=(14, 14), stride=(14, 14))
    (norm): Identity()
  )
  (pos_drop): Dropout(p=0.0, inplace=False)
  (rope): RotaryEmbeddingCat()
  (blocks): ModuleList(
    (0-11): 12 x EvaBlock(
      (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (attn): EvaAttention(
        (q_proj): Linear(in_features=768, out_features=768, bias=True)
        (k_proj): Linear(in_features=768, out_features=768, bias=False)
        (v_proj): Linear(in_features=768, out_features=768, bias=True)
        (attn_drop): Dropout(p=0.0, inplace=False)
        (norm): Identity()
        (proj): Linear(in_features=768, out_features=768, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (drop_path1): Identity()
      (norm2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (mlp): SwiGLU(
        (fc1_g): Linear(in_features=768, out_features=2048, bias=True)
        (fc1_x): Linear(in_features=76

In [20]:
from torchvision import transforms

vit_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3)
])

# Load labels

In [1]:
with open('model/[TMP] labels.txt') as f:
    labels = {i: line.strip().split(' ', 1)[1] for i, line in enumerate(f)}

print(labels)

{0: 'Kek Lapis', 1: 'Kuih Kaswi Pandan', 2: 'Kuih Ketayap', 3: 'Kuih Lapis', 4: 'Kuih Seri Muka', 5: 'Kuih Talam', 6: 'Kuih Ubi Kayu', 7: 'Onde-Onde'}


# Load test images

In [22]:
import os

test_images = [os.path.join(r, f) for r, d, fs in os.walk('test_folder/test_images') for f in fs if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

# CNN predict

In [23]:
from collections import Counter

num_classes = 8
cnn_predictions = []

for path in test_images:
    cnn_results = cnn(path, verbose=False)
    cnn_pred_classes = cnn_results[0].boxes.cls.tolist()  # List of detected class IDs

    if len(cnn_pred_classes) > 0:
        cnn_counts = Counter(cnn_pred_classes)
        cnn_probs = [cnn_counts.get(i, 0) for i in range(num_classes)]
        cnn_probs = [p / sum(cnn_probs) for p in cnn_probs]  # normalize to sum to 1
        cnn_predicted_class_idx = int(max(cnn_counts, key=cnn_counts.get))
    else:
        cnn_probs = [0.0] * num_classes
        cnn_predicted_class_idx = -1

    cnn_predictions.append({
        'image': os.path.basename(path),
        'predicted_class_index': cnn_predicted_class_idx,
        'predicted_label': labels.get(cnn_predicted_class_idx, 'Unknown'),
        'class_probabilities': cnn_probs
    })

# ViT predict

In [24]:
import torch.nn.functional as F
from PIL import Image

vit_predictions = []

for path in test_images:
    image = Image.open(path).convert("RGB")
    input_tensor = vit_transform(image).unsqueeze(0).cuda()

    with torch.no_grad():
        vit_logits = vit(input_tensor)

    vit_probs_tensor = F.softmax(vit_logits, dim=1).squeeze(0)
    vit_probs = vit_probs_tensor.tolist()

    vit_predicted_class_idx = int(torch.argmax(vit_probs_tensor).item())

    vit_predictions.append({
        'image': os.path.basename(path),
        'predicted_class_index': vit_predicted_class_idx,
        'predicted_label': labels.get(vit_predicted_class_idx, 'Unknown'),
        'class_probabilities': vit_probs
    })

print(vit_predictions)

[{'image': 'Kek Lapis.jpg', 'predicted_class_index': 0, 'predicted_label': 'Kek Lapis', 'class_probabilities': [0.9998119473457336, 2.198139191023074e-05, 2.3216711269924417e-05, 3.766033478314057e-05, 2.7468433472677134e-05, 2.6428735509398393e-05, 2.85113765130518e-05, 2.2864156562718563e-05]}, {'image': 'Kuih Kaswi Pandan.jpg', 'predicted_class_index': 1, 'predicted_label': 'Kuih Kaswi Pandan', 'class_probabilities': [2.219474481535144e-05, 0.9998972415924072, 1.8593278582557105e-05, 2.008783667406533e-05, 1.4045052012079395e-05, 1.1336949683027342e-05, 1.0026218660641462e-05, 6.448815383919282e-06]}, {'image': 'Kuih Ketayap.jpg', 'predicted_class_index': 2, 'predicted_label': 'Kuih Ketayap', 'class_probabilities': [1.139003779826453e-05, 1.3869734175386839e-05, 0.9999198913574219, 9.086344107345212e-06, 1.0420495527796447e-05, 1.1227072718611453e-05, 9.811956260818988e-06, 1.4303744137578178e-05]}, {'image': 'Kuih Lapis.jpg', 'predicted_class_index': 3, 'predicted_label': 'Kuih Lap

# Combine model outputs and decide on one class

In [25]:
final_predictions = []

for cnn, vit in zip(cnn_predictions, vit_predictions):
    assert cnn['image'] == vit['image'], "Mismatch in image order"

    cnn_idx = cnn['predicted_class_index']
    vit_idx = vit['predicted_class_index']
    cnn_probs = cnn['class_probabilities']
    vit_probs = vit['class_probabilities']

    #average probabilities (soft voting)
    final_probs = [(c + v) / 2 for c, v in zip(cnn_probs, vit_probs)]

    # Decide class index from final_probs
    if cnn_idx == vit_idx and cnn_idx != -1: #models cohere
        final_idx = cnn_idx

    elif cnn_idx == -1 and vit_idx != -1: #CNN no output, ViT has output, follow ViT
        final_idx = vit_idx

    elif vit_idx == -1 and cnn_idx != -1: #ViT no output, CNN has output, follow CNN
        final_idx = cnn_idx

    elif cnn_idx != -1 and vit_idx != -1: #models disagree, compare confidences from both models using soft voting
        final_idx = int(torch.tensor(final_probs).argmax().item())
    else:
        final_idx = -1  #both models no output

    final_predictions.append({
        'image': cnn['image'],
        'predicted_class_index': final_idx,
        'predicted_label': labels.get(final_idx, 'Unknown'),
        'class_probabilities': final_probs
    })

# Save results to CSV

In [26]:
import pandas as pd

final_results_df = pd.DataFrame(final_predictions)
print(final_results_df)

                   image  predicted_class_index    predicted_label  \
0          Kek Lapis.jpg                      0          Kek Lapis   
1  Kuih Kaswi Pandan.jpg                      1  Kuih Kaswi Pandan   
2       Kuih Ketayap.jpg                      2       Kuih Ketayap   
3         Kuih Lapis.jpg                      3         Kuih Lapis   
4     Kuih Seri Muka.png                      4     Kuih Seri Muka   
5         Kuih Talam.jpg                      5         Kuih Talam   
6      Kuih Ubi Kayu.jpg                      6      Kuih Ubi Kayu   
7          Onde-Onde.jpg                      7          Onde-Onde   

                                 class_probabilities  
0  [0.9999059736728668, 1.099069595511537e-05, 1....  
1  [1.109737240767572e-05, 0.9999486207962036, 9....  
2  [5.695018899132265e-06, 6.934867087693419e-06,...  
3  [2.920833503594622e-05, 1.5045849067973904e-05...  
4  [7.160523182392353e-06, 8.08121603768086e-06, ...  
5  [6.583819867955754e-06, 6.0684060372

# Map the true labels of the images in the testing folder to model's class index

In [27]:
df_test = pd.read_csv('test_folder/test_labels.txt')

filename_to_class = dict(zip(df_test['Filename'].str.strip(), df_test['Class'].str.strip()))

label_to_class_index = {v.strip(): k for k, v in labels.items()}

true_class_index = [label_to_class_index[filename_to_class[os.path.basename(path).strip()]] for path in test_images]

In [28]:
from sklearn.metrics import accuracy_score, roc_auc_score, precision_recall_fscore_support
from sklearn.preprocessing import label_binarize
import numpy as np

final_results_df['true_class_index'] = true_class_index  ## changes here in v2 - refer to true_class_index
y_true = final_results_df['true_class_index'].astype(int).values
y_pred = final_results_df['predicted_class_index'].astype(int).values
y_probs = np.array(final_results_df['class_probabilities'].tolist())

n_classes=8

# Pad probability vectors to length 8
def pad_probs(probs, target_len=n_classes):
    padded = np.zeros(target_len)
    padded[:len(probs)] = probs  # assumes probs are in order (class 0, 1, 2, ...)
    return padded

# Apply padding
y_probs_padded = np.array([pad_probs(p, n_classes) for p in final_results_df['class_probabilities']])

# Update your DataFrame or use directly in metrics
y_probs = y_probs_padded


In [30]:
# Number of classes
class_names = list(range(n_classes))

# Accuracy
acc = accuracy_score(y_true, y_pred)
print(f"\n✅ Accuracy: {acc:.4f}")

# Precision, Recall, F1 per class & macro
prec, rec, f1, _ = precision_recall_fscore_support(y_true, y_pred, labels=class_names, average=None)
macro_prec, macro_rec, macro_f1, _ = precision_recall_fscore_support(y_true, y_pred, average='macro')

print("\n📊 Per-class metrics:")
for i, cls in enumerate(class_names):
    print(f"Class {cls}: Precision={prec[i]:.4f}, Recall={rec[i]:.4f}, F1={f1[i]:.4f}")

print(f"\n📦 Macro Precision: {macro_prec:.4f}, Macro Recall: {macro_rec:.4f}, Macro F1: {macro_f1:.4f}")

# ROC AUC (requires binarized labels)
y_true_bin = label_binarize(y_true, classes=class_names)

# ROC AUC per class and macro
try:
    auc_per_class = roc_auc_score(y_true_bin, y_probs, average=None, multi_class='ovr')
    auc_macro = roc_auc_score(y_true_bin, y_probs, average='macro', multi_class='ovr')

    print("\n🎯 ROC AUC per class:")
    for i, cls in enumerate(class_names):
        print(f"Class {cls}: AUC = {auc_per_class[i]:.4f}")

    print(f"\n🌐 Macro ROC AUC: {auc_macro:.4f}")

except Exception as e:
    print(f"⚠️ ROC AUC could not be computed: {e}")



✅ Accuracy: 1.0000

📊 Per-class metrics:
Class 0: Precision=1.0000, Recall=1.0000, F1=1.0000
Class 1: Precision=1.0000, Recall=1.0000, F1=1.0000
Class 2: Precision=1.0000, Recall=1.0000, F1=1.0000
Class 3: Precision=1.0000, Recall=1.0000, F1=1.0000
Class 4: Precision=1.0000, Recall=1.0000, F1=1.0000
Class 5: Precision=1.0000, Recall=1.0000, F1=1.0000
Class 6: Precision=1.0000, Recall=1.0000, F1=1.0000
Class 7: Precision=1.0000, Recall=1.0000, F1=1.0000

📦 Macro Precision: 1.0000, Macro Recall: 1.0000, Macro F1: 1.0000

🎯 ROC AUC per class:
Class 0: AUC = 1.0000
Class 1: AUC = 1.0000
Class 2: AUC = 1.0000
Class 3: AUC = 1.0000
Class 4: AUC = 1.0000
Class 5: AUC = 1.0000
Class 6: AUC = 1.0000
Class 7: AUC = 1.0000

🌐 Macro ROC AUC: 1.0000
